# Geocoding Police Fire Departments and Mapping to CSI360 Customer Locations

- Import and data from CSI360 Police and Fire List.
- Geocode addresses to obtain latitude and longitude coordinates.
- Insert latitude and longitude columns into data file.
- Check each department location and identify if it is within 100 miles of the customer locations.
- Separate the dataframe for departments within 100 miles for each customer location.
- Export the data to CSV files for further analysis and reporting.

In [ ]:
# Dependencies and Setup
import pandas as pd
from geopy.geocoders import GoogleV3
from geopy.distance import geodesic
# import folium # Use to create html map
# from folium import plugins # Use to create html map
from ratelimit import limits, sleep_and_retry
import time

# from dotenv import load_dotenv # remove if "with open..." works
import os

# Manually load API key from .env file. # Remove code associated with dotenv if this works
with open('C:/Users/jchan/csi360_fire_police/lefd-custy-targets/.env', 'r') as f:
    for line in f:
        if line.startswith('GOOGLE_MAPS_API_KEY'):
            google_api_key = line.split('=')[1].strip()
            # print(f"Loaded API Key: {google_api_key}")


# load_dotenv(".env")  # remove if "with open..." works
# google_api_key = os.getenv('GOOGLE_MAPS_API_KEY') # remove if "with open..." works

In [ ]:
# Store filepath in a variable
lefd_1_data = 'resources/data/lefd_1_data.csv'
csi360_customers = 'resources/data/csi360_customers.csv'

# Read each of the respective files (police, fire, agency_n, agency_addrs) and store into Pandas dataframe
lefd_1_data_df = pd.read_csv(lefd_1_data)
csi360_customers_df = pd.read_csv(csi360_customers)

# Convert the 'zip' column to integer, then back to string to remove decimals
csi360_customers_df['zip'] = csi360_customers_df['zip'].fillna("").apply(lambda x: str(int(float(x))) if x != "" else "")

display(lefd_1_data_df.head(), csi360_customers_df.head())

In [ ]:
## Geocode CSI360 customer zip codes

In [ ]:
# Initialize geolocator. 
geolocator = GoogleV3(api_key=google_api_key)

In [ ]:
# Function to get geocode based on zip code

def get_lat_long(zip_code):
    if zip_code == "":  # Skip empty ZIP codes
        return None, None
    try:
        # Use the ZIP code directly to get the location
        location = geolocator.geocode(zip_code)
        if location:
            return location.latitude, location.longitude
        else:
            print(f"Geocoding failed for ZIP code: {zip_code}")  # Log failed geocodes
            return None, None
    except Exception as e:
        print(f"Error geocoding {zip_code}: {e}")
        return None, None

# Apply the geocode function to get Latitude and Longitude for each row in the cleaned DataFrame
csi360_customers_df['Latitude'], csi360_customers_df['Longitude'] = zip(*csi360_customers_df['zip'].apply(get_lat_long))

# Fill NaN values in 'Latitude' and 'Longitude' with empty strings
csi360_customers_df['Latitude'] = csi360_customers_df['Latitude'].fillna("")
csi360_customers_df['Longitude'] = csi360_customers_df['Longitude'].fillna("")

# Print the first few rows to verify
print(csi360_customers_df.head())


## Geocode Colorado Fire Deptartment Addresses


In [ ]:
# Function to get latitude and longitude from address
# Set rate limit to 25 requests per second
# Google geocoding API allows around 50 queries per second, I've adjusted to 25 to be safe
RATE_LIMIT = 25  # requests per second
TIME_PERIOD = 1  # time period in seconds

# Function to get latitude and longitude from address with rate limiter
@sleep_and_retry
@limits(calls=RATE_LIMIT, period=TIME_PERIOD)
def get_lat_long(address):
    try:
        location = geolocator.geocode(address)
        if location:
            return location.latitude, location.longitude
        else:
            return None, None
    except Exception as e:
        print(f"Error geocoding {address}: {e}")
        return None, None

In [ ]:
# Test with a known address
location = geolocator.geocode("1600 Pennsylvania Ave NW, Washington, DC 20500")
print(location.latitude, location.longitude)

In [ ]:
# Combine address fields and get latitude/longitude
lefd_1_data_df['Full_Address'] = lefd_1_data_df['addr1'] + ', ' + lefd_1_data_df['city'] + ', ' + lefd_1_data_df['state'] + ' ' + lefd_1_data_df['zip'].astype(str)
lefd_1_data_df['Latitude'], lefd_1_data_df['Longitude'] = zip(*lefd_1_data_df['Full_Address'].apply(get_lat_long))
print(lefd_1_data_df.head())